In [ ]:
import matplotlib as plt
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from urllib.request import Request as Req
from bs4 import BeautifulSoup as soup

In [ ]:
keyWord = 'Data-Science'
urlorig='https://www.monster.com/jobs/search/?q='+keyWord+'&where=USA'
urlorig

In [ ]:
titles = []
company_names = []
location_names = []
job_descriptions = []
job_ids = []
refs=[]

global_ind=0

# defining parser to call for each new page
def monster_parser(url):
    global global_ind
    request = Req(url)
    request.add_header('Cache-Control', 'no-cache')
    uClient = uReq(request)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,'html.parser')
    jobs = page_soup.select('section.card-content')
    
    local_ind=0 #change to 1 and check result
    for job in jobs:
        if job.get('data-jobid')!=None:
            if local_ind < global_ind:
                local_ind+=1
            else:
                local_ind+=1
                global_ind+=1
                summaries = job.div.find_all('div', class_='summary')
                summary=summaries[0]

                try:
                    #check if vacancy exist
                    tmp_client = uReq(summary.header.h2.a['href'])
                    tmp_client.close()

                    print(global_ind, local_ind, summary.header.h2.a['href'])

                    #fetch Job ID
                    try:
                        job_ids.append(job['data-jobid'])
                    except:
                        job_ids.append('NA')

                    #fetching URLs
                    try:
                        refs.append(summary.header.h2.a['href'])
                    except:
                        refs.append('NA')

                    #fetch title
                    try:
                        titles.append(summary.header.h2.a.text.strip())
                    except:
                        titles.append('NA')
                    #fetch company
                    try:
                        company_names.append(summary.div.span.text)
                    except:
                        company_names.append('NA')

        #             locations = summary.find_all('div', class_='location')
        #             location = locations[0]
        #             #fetch location
        #             try:
        #                 location_names.append(location.span.text.strip())
        #             except:
        #                 location_names.append('NA')
                    #parse job description and details
                    monster_details_parser(summary.header.h2.a['href'])

                except:
                     print('SKIP - ', summary.header.h2.a['href'])
                    
    next_page = page_soup.select('div.mux-search-results a#loadMoreJobs.mux-btn.btn-secondary')
    next_url = next_page[0]['href']+'&q=Data-Science&where=USA'
    monster_parser(next_url)
                
def monster_details_parser(url):
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,'html.parser')
    
    descr = page_soup.select('div#JobDescription')
    try:
        job_descriptions.append(descr[0].text)
    except:
        job_descriptions.append('NA')
        
    values = page_soup.select('div.mux-job-summary section.summary-section dl.header dd.value')
    for value in values:
        if value.parent.dt.text == "Location":
            #fetch location
            try:
                location_names.append(value.text)
            except:
                location_names.append('NA')


In [ ]:
my_url = urlorig

monster_parser(my_url)

In [ ]:
#single page parser testing
monster_details_parser('https://job-openings.monster.com/principal-data-scientist-facilities-analytics-redondo-beach-ca-us-northrop-grumman/58367067-6657-41c9-abb6-46a6aefdf6eb')

In [ ]:
print(refs)